<a href="https://colab.research.google.com/github/NULabTMN/ps3-PoojaKShanbhag/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. I cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [0]:
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy in c:\users\pooja\anaconda3\envs\python36cudapytorch\lib\site-packages (2.2.4)
Requirement already up-to-date: allennlp in c:\users\pooja\anaconda3\envs\python36cudapytorch\lib\site-packages (0.9.0)


2.2.4


In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [0]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

562it [00:00, 10806.64it/s]
23it [00:00, 1915.32it/s]
100%|█████████████████████████████████████████████████████████████████████████████| 585/585 [00:00<00:00, 73080.20it/s]


## Define and Train Model

In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9018 ||: 100%|█████████████████████████████████████████████| 281/281 [00:04<00:00, 58.59it/s]
accuracy: 0.7878, loss: 1.2140 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 139.37it/s]
accuracy: 0.8442, loss: 0.7298 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 72.63it/s]
accuracy: 0.7878, loss: 1.1818 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 168.68it/s]
accuracy: 0.8442, loss: 0.7166 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 78.61it/s]
accuracy: 0.7878, loss: 1.1876 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 172.57it/s]
accuracy: 0.8442, loss: 0.7061 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 86.29it/s]
accuracy: 0.7878, loss: 1.1596 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 190.19it/s]
accuracy: 0.8442, loss: 0.6974 ||: 100%|

accuracy: 0.8770, loss: 0.3403 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 75.44it/s]
accuracy: 0.8082, loss: 0.6726 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 150.00it/s]
accuracy: 0.8772, loss: 0.3370 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 79.81it/s]
accuracy: 0.8163, loss: 0.6493 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 176.48it/s]
accuracy: 0.8786, loss: 0.3354 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 80.00it/s]
accuracy: 0.8224, loss: 0.6428 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 166.67it/s]
accuracy: 0.8779, loss: 0.3322 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 78.87it/s]
accuracy: 0.8184, loss: 0.6495 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 173.75it/s]
accuracy: 0.8799, loss: 0.3297 ||: 100%|

accuracy: 0.9046, loss: 0.2528 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 91.71it/s]
accuracy: 0.8551, loss: 0.5082 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 166.66it/s]
accuracy: 0.9093, loss: 0.2481 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 91.13it/s]
accuracy: 0.8612, loss: 0.4979 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 187.50it/s]
accuracy: 0.9094, loss: 0.2459 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 86.91it/s]
accuracy: 0.8592, loss: 0.4938 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 187.50it/s]
accuracy: 0.9095, loss: 0.2436 ||: 100%|█████████████████████████████████████████████| 281/281 [00:03<00:00, 76.49it/s]
accuracy: 0.8633, loss: 0.4819 ||: 100%|██████████████████████████████████████████████| 12/12 [00:00<00:00, 187.50it/s]
accuracy: 0.9119, loss: 0.2428 ||: 100%|

{'best_epoch': 99,
 'peak_cpu_memory_MB': 0,
 'peak_gpu_0_memory_MB': 615,
 'training_duration': '0:05:55.246723',
 'training_start_epoch': 0,
 'training_epochs': 99,
 'epoch': 99,
 'training_accuracy': 0.9277618896303358,
 'training_loss': 0.18980582684276,
 'training_cpu_memory_MB': 0.0,
 'training_gpu_0_memory_MB': 615,
 'validation_accuracy': 0.8816326530612245,
 'validation_loss': 0.3906479166083348,
 'best_validation_accuracy': 0.8816326530612245,
 'best_validation_loss': 0.3906479166083348}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
    tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
    fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    return list(fields)
tag_sentence(validation_dataset[0])

baseline_output = [tag_sentence(i) for i in validation_dataset]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [0]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
    count = 0
#     sen_index = 0
    for sentence in tagged:
#         sen_index += 1
#         index = 0
        prev_tag = ''
#         print("\n")
#         print("sentence "+str(sen_index))
        for word in sentence:
#             index += 1
#             print(word," - "*10,str(index))
            tag = word[2]
            if tag != 'O':
                if tag.startswith('I-'):
                    if prev_tag == '':
                        count += 1
                    else:
                        if tag[2:] != prev_tag:
                            count += 1
                    prev_tag = tag[2:]
                elif tag.startswith('B-'):
                    prev_tag = tag[2:]
            else:
                prev_tag = ''
                
            
    return count


# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
    gold_span, model_span = list(), list()
    sen_index = 0
    for sentence in tagged:
        sen_index += 1
        
        prev_tag_gold = ''
        word_index = 0
        start_gold = 0
        end_gold = 0
        prev_tag_model = ''
        start_model, end_model = 0,0 
        for word in sentence:
            word_index += 1
            tag_gold = word[1]
            if tag_gold != 'O':
                if tag_gold.startswith("B-"):
                    end_gold = word_index - 1
                    if prev_tag_gold != '':
                        gold_span.append((prev_tag_gold, start_gold, end_gold, sen_index))
                    start_gold = word_index
                    prev_tag_gold = tag_gold[2:]
            else: 
                end_gold = word_index - 1
                if prev_tag_gold != '':
                    gold_span.append((prev_tag_gold, start_gold, end_gold, sen_index))
                prev_tag_gold = ''
                                         
            tag_model = word[2]
                                    
            if tag_model != 'O':
                if tag_model.startswith("B-"):
                    end_model = word_index - 1
                    if prev_tag_model != '':
                        model_span.append((prev_tag_model, start_model, end_model, sen_index))
                    start_model = word_index
                    prev_tag_model = tag_model[2:]
                elif tag_model.startswith("I-"):
                    if prev_tag_model != tag_model[2:]:
                        prev_tag_model = ''
                        
            else:
                end_model = word_index - 1
                if prev_tag_model != '':
                    model_span.append((prev_tag_model, start_model, end_model, sen_index))
                prev_tag_model = ''
        
        if prev_tag_gold != '':
            end_gold = word_index - 1
            gold_span.append((prev_tag_gold, start_gold, end_gold, sen_index))
            
        if prev_tag_model != '':
            end_model = word_index - 1
            model_span.append((prev_tag_model, start_model, end_model, sen_index))
    
#     print(gold_span)
#     print('-'*15)
#     print(model_span)  
    
    matching_span_count = 0 
    for each in model_span:
        if each in gold_span:
            matching_span_count += 1
            
#     print(matching_span_count)
    
    precision = matching_span_count/ len(model_span)
    recall = matching_span_count / len(gold_span)
    
    f1 = 2*(precision * recall)/ (precision+recall)
#     print(precision, " ", recall, " ", f1)
    
#     print(len(gold_span)," ", len(model_span))
    return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
violations(baseline_output)

32

In [0]:
span_stats(baseline_output)

{'precision': 0.5555555555555556,
 'recall': 0.23255813953488372,
 'f1': 0.3278688524590164}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [0]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [0]:
n_inf = float("-inf")
transition_matrix = list()

initial_prob = []
vocab_dict = vocab.get_index_to_token_vocabulary('labels')
for index in vocab_dict:
    label = vocab_dict[index]
    if label == 'O' or label.startswith("B"):
        initial_prob.append(0)
    else:
        initial_prob.append(n_inf)

for row_i in vocab_dict:
    row_label = vocab_dict[row_i]
    cur_tag = ''
    if row_label != 'O':
        cur_tag = row_label[2:]
    new_list = list()
#     new_list.append(row_label)
    for col_i in vocab_dict:
        col_label = vocab_dict[col_i]
        if row_label == 'O':
            if col_label =='O' or col_label.startswith("B"):
                new_list.append(0)
            else:
                new_list.append(n_inf)
        else:
            if col_label.startswith("B") or (col_label.startswith("I") and cur_tag == col_label[2:]) or col_label == 'O':
                new_list.append(0)
            else:
                new_list.append(n_inf)
    transition_matrix.append(new_list)
trans_prob = np.array(transition_matrix)

In [0]:
def viterbi_decoder(s):
    obs_likelyhood = predictor.predict_instance(s)['tag_logits']
    emission_prob = np.array(obs_likelyhood)
    
    T = len(emission_prob)
    N = len(vocab_dict)
    
    viterabi = np.array([[0.0 for j in range(T)] for i in range(N)])
    backpointers = np.array([[-1 for j in range(T)] for i in range(N)])
    
    for i in range(T):
        if i==0:
            viterabi[:,0]= np.add(initial_prob, emission_prob[i])
        else:
            for j in range(N):
                viterabi[j,i]= np.max(np.add(trans_prob[:,j] , viterabi[:,i-1])) + emission_prob[i,j]
                backpointers[j,i] = np.argmax(np.add(trans_prob[:,j] , viterabi[:,i-1]))
        
#     print(viterabi)
#     print(type(backpointers))
    ind = np.argmax(viterabi[:,i])
    tag_ids = []
    while ind != -1:
        tag_ids.append(ind)
        ind = backpointers[ind,i]
        i -= 1

    tag_ids.reverse()
#     print(tag_ids)
    fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    return list(fields)
    
    
baseline_viterbi_outputs = [viterbi_decoder(i) for i in validation_dataset]    

In [0]:
violations(baseline_viterbi_outputs)

0

In [0]:
span_stats(baseline_viterbi_outputs)

{'precision': 0.3829787234042553, 'recall': 0.4186046511627907, 'f1': 0.4}